### Imports

In [1]:
import shutil
import csv
import datetime
import numpy as np
import pandas as pd
from sklearn.metrics import recall_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from packages.metagenomics import sampling2, encoding2
from sklearn.linear_model import LogisticRegression

from packages.LogisticRegression.MulticlassLogisticRegression import MulticlassLogisticRegression,MulticlassLogisticRegression2


### Ideas for improvements
- try converting back to sparse matrix after adding augmented column?

In [2]:
def append_results_to_file(filename, fields=None, rows=None):
    
    with open(filename, 'a') as f:

        write = csv.writer(f)

        if fields:
            write.writerow(fields)

        if rows:
            write.writerows(rows)

In [3]:
def run_mlr_classification_recall(X_train, X_test, y_train, y_test, eta, epsilon):
    """
    Score is species level recall.
    """
    mlr = MulticlassLogisticRegression2(eta=eta, epsilon=epsilon)
    mlr.fit(X_train,y_train)
    y_pred = mlr.predict(X_test)
    score = recall_score(y_test, y_pred, average='weighted')
    return score
    

In [4]:
def build_fragments(seq_file, taxid_file, output_dir, sample_length, coverage, seed):
    # delete output directory if it previously exists
    try:
        shutil.rmtree(output_dir)
    except FileNotFoundError:
        print('Existing directory was not found. Process will generate a directory.')

    # build fragments
    sampling2.generate_fragment_data(seq_file, taxid_file, output_dir, sample_length, coverage, seed)

In [5]:
def encode_fragments(output_dir, pattern, k, seed):
    
    # encode data
    fragments = sampling2.read_fragments(output_dir, pattern)
    X_enc, y = encoding2.encode_fragment_dataset(fragments,k)
    le = preprocessing.LabelEncoder()
    y_enc = le.fit_transform(y)
    
    # calculate number of classes
    n_classes = len(np.unique(y_enc))
#     print('n_classes:',n_classes)
    n_classes_train = 0
    n_classes_test = 0
    while n_classes_train < n_classes or n_classes_test < n_classes:

        # split data into test and training
        X_train, X_test, y_train, y_test = train_test_split(X_enc.toarray(), y_enc, test_size=0.33)
        n_classes_train = len(np.unique(y_train))
        n_classes_test = len(np.unique(y_test))
#         print('train:',len(y_train))
#         print('test:', len(y_test))
    
    print(X_enc.shape)
    
    return X_train, X_test, y_train, y_test

In [6]:
def calc_number_combinations(list_sample_length,list_coverage,list_k,list_eta,list_epsilon):
    n = len(list_sample_length) * len(list_coverage) * len(list_k) *len(list_eta) * len(list_epsilon)
    return n

In [7]:
def grid_search_multiclass_mlr(seq_file, 
                              taxid_file, 
                              output_dir, 
                              pattern, 
                              list_sample_length, 
                              list_coverage, 
                              list_k,
                              list_eta,
                              list_epsilon, 
                              seed,
                              grid_search_file,
                              fields,
                              experiment,
                              score_type):
    
    # set up grid search results file
    append_results_to_file(grid_search_file, fields)
    
    # calculate number of combinations
    n_combinations = calc_number_combinations(list_sample_length,list_coverage,list_k,list_eta,list_epsilon)
    
    # process combinations
    count = 0
    for sample_length in list_sample_length:
        for coverage in list_coverage:
            
            # fragment combination
            build_fragments(seq_file, taxid_file, output_dir, sample_length, coverage, seed)
            for k in list_k:
                
                # kmer combination
                X_train, X_test, y_train, y_test = encode_fragments(output_dir, pattern,k,seed)
                for eta in list_eta:
                    for epsilon in list_epsilon:
                        
                        # random forest combination
                        score = run_mlr_classification_recall(X_train, X_test, y_train,y_test, eta, epsilon)
                        count += 1
                        
                        # output results to file
                        row = [experiment, 'multiclass', 'Logistic Regression', X_train.shape, sample_length, coverage, k, eta, epsilon, score, score_type]
                        append_results_to_file(grid_search_file, row)
                        
                print('Percent complete: {}'.format(count / n_combinations * 100)) # display progress
    

# Run Set 1 - MLR Toy
2000 lengths dataset

### Run 4.01
Stopped early due to runs taking a long time.

In [8]:
# %%time

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000-mlr'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/runs-2000/mlr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','eta', 'epsilon', 'score','score type']
# experiment = '4.01'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [100]
# list_coverage = [0.2,1,10]
# list_k = [1,2,4,6,8,10,12]
# list_eta = [0.01]
# list_epsilon = [0.01]


# grid_search_multiclass_mlr(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_eta,
#                           list_epsilon,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)


### Run 4.02

In [9]:
# %%time

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000-mlr'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/runs-2000/mlr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','eta', 'epsilon', 'score','score type']
# experiment = '4.02'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [200]
# list_coverage = [0.3,1,10]
# list_k = [1,2,4,6,8,10,12]
# list_eta = [0.01]
# list_epsilon = [0.01]


# grid_search_multiclass_mlr(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_eta,
#                           list_epsilon,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)


### Run 4.03

In [10]:
# %%time

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000-mlr'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/runs-2000/mlr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','eta', 'epsilon', 'score','score type']
# experiment = '4.03'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [400]
# list_coverage = [0.5,1,10]
# list_k = [1,2,4,6,8,10,12]
# list_eta = [0.01]
# list_epsilon = [0.01]


# grid_search_multiclass_mlr(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_eta,
#                           list_epsilon,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)


### Run 4.04

In [11]:
# %%time

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000-mlr'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/runs-2000/mlr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','eta', 'epsilon', 'score','score type']
# experiment = '4.04'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [100]
# list_coverage = [100]
# list_k = [1,2,4,6,8,10,12]
# list_eta = [0.01]
# list_epsilon = [0.01]


# grid_search_multiclass_mlr(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_eta,
#                           list_epsilon,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)


### Run 4.05

In [12]:
# %%time
##40 min

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000-mlr'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/runs-2000/mlr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','eta', 'epsilon', 'score','score type']
# experiment = '4.05'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [100]
# list_coverage = [200]
# list_k = [1,2,4,6,8,10,12]
# list_eta = [0.01]
# list_epsilon = [0.01]


# grid_search_multiclass_mlr(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_eta,
#                           list_epsilon,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)


# Run Set 2 - sklearn with l1 penalty
Compare with sklearn implementation of MLR to see if performance is similar.

In [13]:
# def calc_number_combinations2(list_sample_length,list_coverage,list_k,list_multiclass,list_classweight):
#     n = len(list_sample_length) * len(list_coverage) * len(list_k) *len(list_multiclass) * len(list_classweight)
#     return n

In [14]:
def encode_fragments2(output_dir, pattern, k, seed):
    """
    Does not convert sparse matrix to numpy matrix.
    """
    
    # encode data
    fragments = sampling2.read_fragments(output_dir, pattern)
    X_enc, y = encoding2.encode_fragment_dataset(fragments,k)
    le = preprocessing.LabelEncoder()
    y_enc = le.fit_transform(y)
    
    # calculate number of classes
    n_classes = len(np.unique(y_enc))
#     print('n_classes:',n_classes)
    n_classes_train = 0
    n_classes_test = 0
    while n_classes_train < n_classes or n_classes_test < n_classes:

        # split data into test and training
        X_train, X_test, y_train, y_test = train_test_split(X_enc, y_enc, test_size=0.33)
        n_classes_train = len(np.unique(y_train))
        n_classes_test = len(np.unique(y_test))
#         print('train:',y_train)
#         print('test:', y_test)
    
    print(X_enc.shape)
    
    return X_train, X_test, y_train, y_test

In [15]:
# def run_lr_classification_recall(X_train, X_test, y_train, y_test, multiclass, classweight, seed ):
#     """
#     Score is species level recall. Uses sklearn version of logistic regression.
#     """
#     lr = LogisticRegression(random_state=seed, multi_class=multiclass, class_weight=classweight )
#     lr.fit(X_train,y_train)
#     y_pred = lr.predict(X_test)
#     score = recall_score(y_test, y_pred, average='weighted')
#     return score
    

In [16]:
# def grid_search_multiclass_lr(seq_file, 
#                               taxid_file, 
#                               output_dir, 
#                               pattern, 
#                               list_sample_length, 
#                               list_coverage, 
#                               list_k,
#                               list_multiclass,
#                               list_classweight,
#                               seed,
#                               grid_search_file,
#                               fields,
#                               experiment,
#                               score_type):
    
#     # set up grid search results file
#     append_results_to_file(grid_search_file, fields)
    
#     # calculate number of combinations
#     n_combinations = calc_number_combinations2(list_sample_length,list_coverage,list_k, list_multiclass, list_classweight)
    
#     # process combinations
#     count = 0
#     for sample_length in list_sample_length:
#         for coverage in list_coverage:
            
#             # fragment combination
#             build_fragments(seq_file, taxid_file, output_dir, sample_length, coverage, seed)
#             for k in list_k:
                
#                 # kmer combination
#                 X_train, X_test, y_train, y_test = encode_fragments2(output_dir, pattern,k,seed)
                
                
#                 for multiclass in list_multiclass:
#                     for classweight in list_classweight:

                        
#                         # random forest combination
#                         score = run_lr_classification_recall(X_train, X_test, y_train, y_test, multiclass, classweight, seed)
#                         count += 1

#                         # output results to file
#                         row = [experiment, 'multiclass', 'Logistic Regression (sklearn)', X_train.shape, sample_length, coverage, k, multiclass, classweight, score, score_type]
#                         append_results_to_file(grid_search_file, row)

#                 print('Percent complete: {}'.format(count / n_combinations * 100)) # display progress


### Run 5.01

In [17]:
# %%time

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/runs-2000/lr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','multiclass', 'class_weight', 'score','score type']
# experiment = '5.01'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [100,200,400]
# list_coverage = [1,10,100,200,400]
# list_k = [1]
# list_multiclass = ['auto']
# list_classweight= [None]


# grid_search_multiclass_lr(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_multiclass,
#                           list_classweight,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)


### Run 5.02

In [18]:
# %%time

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/runs-2000/lr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','multiclass', 'class_weight', 'score','score type']
# experiment = '5.02'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [100,200,400]
# list_coverage = [1,10,100,200,400]
# list_k = [2,4,6,8,10,12]
# list_multiclass = ['auto']
# list_classweight= [None]


# grid_search_multiclass_lr(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_multiclass,
#                           list_classweight,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)


# Run Set 3 - MLR with L2 penalty
- toy-2000 dataset
- See how performance changes with L2 penalty
- Changed breakpoint in gradient descent to 100 from 100,000 as well.

In [19]:
def calc_number_combinations(*args):
    total = 1
    for each in args:
        total *= len(each)
    return total

In [20]:
def parameter_generator(list_sample_length,list_coverage,list_k):
    
    for L in list_sample_length:
        for c in list_coverage:
            for k in list_k:
                yield L, c, k

In [21]:
def hyperparameter_generator(list_eta,list_epsilon, list_penalty, list_l2_lambda,list_max_iter):
    
    for eta in list_eta:
        for e in list_epsilon:
            for penalty in list_penalty:
                for l2 in list_l2_lambda:
                    for m in list_max_iter:
                        yield eta,e,penalty,l2,m

In [22]:
def run_mlr_classification_recall(X_train, X_test, y_train, y_test, eta, epsilon, penalty, l2_lambda, max_iter):
    """
    Score is species level recall.
    """
    mlr = MulticlassLogisticRegression2(eta=eta, 
                                        epsilon=epsilon, 
                                        penalty=penalty, 
                                        l2_lambda=l2_lambda, 
                                        max_iter=max_iter)
    mlr.fit(X_train,y_train)
    y_pred = mlr.predict(X_test)
    score = recall_score(y_test, y_pred, average='weighted')
    return score
    

In [23]:
def grid_search_multiclass_mlr(seq_file, 
                              taxid_file, 
                              output_dir, 
                              pattern, 
                              list_sample_length, 
                              list_coverage, 
                              list_k,
                              list_eta,
                              list_epsilon, 
                              list_penalty,
                              list_l2_lambda,
                              list_max_iter,
                              seed,
                              grid_search_file,
                              fields,
                              experiment,
                              score_type):
    
    # set up grid search results file
    append_results_to_file(grid_search_file, fields)
    
    # calculate number of combinations
    n_combinations = calc_number_combinations(list_sample_length,
                                              list_coverage,
                                              list_k,
                                              list_eta,
                                              list_epsilon, 
                                              list_penalty,
                                              list_l2_lambda, 
                                              list_max_iter)
    
    # process combinations
    count = 0
    sample_length_prev = -1
    coverage_prev = -1
    
    # parameter combinations
    for sample_length, coverage,k in parameter_generator(list_sample_length,list_coverage,list_k):
        print(sample_length, coverage,k)
        
        if sample_length != sample_length_prev or coverage != coverage_prev:
            
            # fragment combination
            build_fragments(seq_file, taxid_file, output_dir, sample_length, coverage, seed)
            
            # update previous values
            sample_length_prev = sample_length
            coverage_prev = coverage
                
        # kmer from fragments
        X_train, X_test, y_train, y_test = encode_fragments(output_dir, pattern,k,seed)
        
        
        # hyperparameter combinations
        for eta, epsilon, penalty, l2_lambda, max_iter in hyperparameter_generator(list_eta,list_epsilon, list_penalty, list_l2_lambda,list_max_iter):
            print(eta, epsilon, penalty, l2_lambda, max_iter)
            
            # train and score model
            score = run_mlr_classification_recall(X_train, X_test, y_train, y_test, eta, epsilon, penalty, l2_lambda, max_iter)
            count += 1

            # output results to file
            row = [experiment, 'multiclass', 'Logistic Regression', X_train.shape, sample_length, coverage, k, eta, epsilon, penalty, l2_lambda, max_iter, score, score_type]
            append_results_to_file(grid_search_file, row)
                        
        print('Percent complete: {}'.format(count / n_combinations * 100)) # display progress
    

### Run 6.01

In [24]:
# %%time

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000-mlr'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/runs-2000/mlr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','eta', 'epsilon', 'l2_penalty','score','score type']
# experiment = '6.01'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [100]
# list_coverage = [1,10,100,200,400]
# list_k = [1,2,4,6,8,10,12]
# list_eta = [0.01]
# list_epsilon = [0.01]
# list_l2_penalty = [1,10,100, 0.1]


# grid_search_multiclass_mlr_l2(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_eta,
#                           list_epsilon,
#                           list_l2_penalty,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)


### Run 6.02
Testing out range for lambda. It appears that close to zero (0, 0.1) is best, with values decreasing more after 1.0.

In [25]:
# %%time

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/gridsearch-2000/mlr-multi.{}.csv'.format(date_time)
# fields = ['experiment',
#           'category',
#           'classifier',
#           'training shape',
#           'sample_length',
#           'coverage',
#           'k',
#           'eta', 
#           'epsilon', 
#           'penalty',
#           'l2_lambda',
#           'max_iter',
#           'score',
#           'score type']

# experiment = '6.02'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [100]
# list_coverage = [200]
# list_k = [4]
# list_eta = [0.01]
# list_epsilon = [0.01]
# list_penalty = ['l2']
# list_l2_lambda = [0,0.1,0.5,1,3,6,10]
# list_max_iter = [100]


# grid_search_multiclass_mlr(seq_file, 
#                                 taxid_file, 
#                                 output_dir, 
#                                 pattern, 
#                                 list_sample_length, 
#                                 list_coverage, 
#                                 list_k, 
#                                 list_eta,
#                                 list_epsilon,
#                                 list_penalty,
#                                 list_l2_lambda,
#                                 list_max_iter,
#                                 seed,
#                                 grid_search_file,
#                                 fields,
#                                 experiment,
#                                 score_type)


### Run 6.03
Finishing out data for plotting

In [26]:
# %%time
# 5.5 h

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/gridsearch-2000/mlr-multi.{}.csv'.format(date_time)
# fields = ['experiment',
#           'category',
#           'classifier',
#           'training shape',
#           'sample_length',
#           'coverage',
#           'k',
#           'eta', 
#           'epsilon', 
#           'penalty',
#           'l2_lambda',
#           'max_iter',
#           'score',
#           'score type']

# experiment = '6.03'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [100]
# list_coverage = [400]
# list_k = [1,2,4,6,8,10,12]
# list_eta = [0.01]
# list_epsilon = [0.01]
# list_penalty = [None]
# list_l2_lambda = [0]
# list_max_iter = [100]


# grid_search_multiclass_mlr(seq_file, 
#                                 taxid_file, 
#                                 output_dir, 
#                                 pattern, 
#                                 list_sample_length, 
#                                 list_coverage, 
#                                 list_k, 
#                                 list_eta,
#                                 list_epsilon,
#                                 list_penalty,
#                                 list_l2_lambda,
#                                 list_max_iter,
#                                 seed,
#                                 grid_search_file,
#                                 fields,
#                                 experiment,
#                                 score_type)


### Run 6.04
Finishing out data for plotting
Try a larger learning rate to see if that speeds up processing.

In [27]:
# %%time
# # 4 h 32 min

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/gridsearch-2000/mlr-multi.{}.csv'.format(date_time)
# fields = ['experiment',
#           'category',
#           'classifier',
#           'training shape',
#           'sample_length',
#           'coverage',
#           'k',
#           'eta', 
#           'epsilon', 
#           'penalty',
#           'l2_lambda',
#           'max_iter',
#           'score',
#           'score type']

# experiment = '6.04'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [200,400]
# list_coverage = [100,200,400]
# list_k = [1,2,4,6,8,10,12]
# list_eta = [0.02]
# list_epsilon = [0.01]
# list_penalty = [None]
# list_l2_lambda = [0]
# list_max_iter = [100]


# grid_search_multiclass_mlr(seq_file, 
#                                 taxid_file, 
#                                 output_dir, 
#                                 pattern, 
#                                 list_sample_length, 
#                                 list_coverage, 
#                                 list_k, 
#                                 list_eta,
#                                 list_epsilon,
#                                 list_penalty,
#                                 list_l2_lambda,
#                                 list_max_iter,
#                                 seed,
#                                 grid_search_file,
#                                 fields,
#                                 experiment,
#                                 score_type)


200 100 1
(4071, 800)
0.02 0.01 None 0 100
n_classifiers 5
training classifier 0
training classifier 1


/Users/ryanqnelson/GitHub/C-A-L-C-I-F-E-R/CMU-02620-Metagenomics/packages/LogisticRegression/gradient_descent.py:303: RuntimeWarning: overflow encountered in exp
  inner = ones + np.exp(Xw)
/Users/ryanqnelson/GitHub/C-A-L-C-I-F-E-R/CMU-02620-Metagenomics/packages/LogisticRegression/gradient_descent.py:254: RuntimeWarning: overflow encountered in exp
  top = np.exp(Xw)
/Users/ryanqnelson/GitHub/C-A-L-C-I-F-E-R/CMU-02620-Metagenomics/packages/LogisticRegression/gradient_descent.py:257: RuntimeWarning: invalid value encountered in true_divide
  return top / bottom


training classifier 2
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 3
training classifier 4
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
Percent complete: 2.380952380952381
200 100 2
(4071, 1600)
0.02 0.01 None 0 100
n_classifiers 5
training classifier 0
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 1
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 2
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 3
training classifier 4
Percent complete: 4.761904761904762
200 100 4
(4071, 12772)
0.02 0.01 None 0 100
n_classifiers 5
training classifier 0
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 1
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 2
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 3
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 4
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
Percent complete: 7.142857142857142
200 100 6
(4071, 60742)
0.02 0.01 N

/Users/ryanqnelson/GitHub/C-A-L-C-I-F-E-R/CMU-02620-Metagenomics/packages/LogisticRegression/gradient_descent.py:303: RuntimeWarning: overflow encountered in exp
  inner = ones + np.exp(Xw)
/Users/ryanqnelson/GitHub/C-A-L-C-I-F-E-R/CMU-02620-Metagenomics/packages/LogisticRegression/gradient_descent.py:254: RuntimeWarning: overflow encountered in exp
  top = np.exp(Xw)
/Users/ryanqnelson/GitHub/C-A-L-C-I-F-E-R/CMU-02620-Metagenomics/packages/LogisticRegression/gradient_descent.py:257: RuntimeWarning: invalid value encountered in true_divide
  return top / bottom


Percent complete: 19.047619047619047
200 200 2
(8141, 1600)
0.02 0.01 None 0 100
n_classifiers 5
training classifier 0
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 1
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 2
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 3
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 4
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
Percent complete: 21.428571428571427
200 200 4
(8141, 12799)
0.02 0.01 None 0 100
n_classifiers 5
training classifier 0
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 1
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 2
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 3
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 4
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
Percent complete: 23.809523809523807
200 200 6
(8141, 77930)
0.02 0.01 None 0 100
n_classifiers 5
training classifier 0
STOP: TOTAL NO.

...

training classifier 2
training classifier 3
training classifier 4
Percent complete: 52.38095238095239
400 100 2
(2037, 3200)
0.02 0.01 None 0 100
n_classifiers 5
training classifier 0
training classifier 1
training classifier 2
training classifier 3
training classifier 4
Percent complete: 54.761904761904766
400 100 4
(2037, 25059)
0.02 0.01 None 0 100
n_classifiers 5
training classifier 0
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 1
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 2
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 3
training classifier 4
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
Percent complete: 57.14285714285714
400 100 6
(2037, 80853)
0.02 0.01 None 0 100
n_classifiers 5
training classifier 0
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 1
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 2
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 3
STOP: TOTAL NO. of I

/Users/ryanqnelson/GitHub/C-A-L-C-I-F-E-R/CMU-02620-Metagenomics/packages/LogisticRegression/gradient_descent.py:303: RuntimeWarning: overflow encountered in exp
  inner = ones + np.exp(Xw)
/Users/ryanqnelson/GitHub/C-A-L-C-I-F-E-R/CMU-02620-Metagenomics/packages/LogisticRegression/gradient_descent.py:254: RuntimeWarning: overflow encountered in exp
  top = np.exp(Xw)
/Users/ryanqnelson/GitHub/C-A-L-C-I-F-E-R/CMU-02620-Metagenomics/packages/LogisticRegression/gradient_descent.py:257: RuntimeWarning: invalid value encountered in true_divide
  return top / bottom


training classifier 4
Percent complete: 69.04761904761905
400 200 2
(4071, 3200)
0.02 0.01 None 0 100
n_classifiers 5
training classifier 0
training classifier 1
training classifier 2
training classifier 3
training classifier 4
Percent complete: 71.42857142857143
400 200 4
(4071, 25514)
0.02 0.01 None 0 100
n_classifiers 5
training classifier 0
training classifier 1
training classifier 2
training classifier 3
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 4
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
Percent complete: 73.80952380952381
400 200 6
(4071, 117598)
0.02 0.01 None 0 100
n_classifiers 5
training classifier 0
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 1
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 2
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 3
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 4
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
Percent complete: 76.19047619047619
400 20

/Users/ryanqnelson/GitHub/C-A-L-C-I-F-E-R/CMU-02620-Metagenomics/packages/LogisticRegression/gradient_descent.py:303: RuntimeWarning: overflow encountered in exp
  inner = ones + np.exp(Xw)
/Users/ryanqnelson/GitHub/C-A-L-C-I-F-E-R/CMU-02620-Metagenomics/packages/LogisticRegression/gradient_descent.py:254: RuntimeWarning: overflow encountered in exp
  top = np.exp(Xw)
/Users/ryanqnelson/GitHub/C-A-L-C-I-F-E-R/CMU-02620-Metagenomics/packages/LogisticRegression/gradient_descent.py:257: RuntimeWarning: invalid value encountered in true_divide
  return top / bottom


training classifier 2
training classifier 3
training classifier 4
Percent complete: 85.71428571428571
400 400 2
(8141, 3200)
0.02 0.01 None 0 100
n_classifiers 5
training classifier 0
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 1
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 2
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 3
training classifier 4
Percent complete: 88.09523809523809
400 400 4
(8141, 25588)
0.02 0.01 None 0 100
n_classifiers 5
training classifier 0
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 1
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 2
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 3
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
training classifier 4
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
Percent complete: 90.47619047619048
400 400 6
(8141, 148288)
0.02 0.01 None 0 100
n_classifiers 5
training classifier 0
STOP: TOTAL NO. of ITERATIONS REACHED LIM

# Run Set 4 - sklearn with l1 penalty
To compare against L2 results

In [28]:
def hyperparameter_generator_lr(list_penalty,list_multiclass,list_classweight):
    
    for penalty in list_penalty:
        for multiclass in list_multiclass:
            for classweight in list_classweight:
                yield penalty,multiclass,classweight

In [29]:
def run_lr_classification_recall(X_train, X_test, y_train, y_test, penalty,multiclass,classweight, seed):
    """
    Score is species level recall.
    Sets solver to 'saga' for l1 penalty. Uses default solver for l2 penalty. solver='saga'
    """
    lr = LogisticRegression(penalty=penalty, multi_class=multiclass, class_weight=classweight,random_state=seed )
    lr.fit(X_train,y_train)
    y_pred = lr.predict(X_test)
    score = recall_score(y_test, y_pred, average='weighted')
    return score
    

In [30]:
def grid_search_multiclass_lr(seq_file, 
                          taxid_file, 
                          output_dir, 
                          pattern, 
                          list_sample_length, 
                          list_coverage, 
                          list_k, 
                          list_penalty,
                          list_multiclass,
                          list_classweight,
                          seed,
                          grid_search_file,
                          fields,
                          experiment,
                          score_type):
    
    # set up grid search results file
    append_results_to_file(grid_search_file, fields)
    
    # calculate number of combinations
    n_combinations = calc_number_combinations(list_sample_length,
                                              list_coverage,
                                              list_k,
                                              list_eta,
                                              list_epsilon, 
                                              list_penalty,
                                              list_l2_lambda, 
                                              list_max_iter)
    
    # process combinations
    count = 0
    sample_length_prev = -1
    coverage_prev = -1
    
    # parameter combinations
    for sample_length, coverage,k in parameter_generator(list_sample_length,list_coverage,list_k):
        print(sample_length, coverage,k)
        
        if sample_length != sample_length_prev or coverage != coverage_prev:
            
            # fragment combination
            build_fragments(seq_file, taxid_file, output_dir, sample_length, coverage, seed)
            
            # update previous values
            sample_length_prev = sample_length
            coverage_prev = coverage
                
        # kmer from fragments
        X_train, X_test, y_train, y_test = encode_fragments2(output_dir, pattern,k,seed)
        
        
        # hyperparameter combinations
        for penalty,multiclass,classweight in hyperparameter_generator_lr(list_penalty,list_multiclass,list_classweight):
            print(penalty,multiclass,classweight)
            
            # train and score model
            score = run_lr_classification_recall(X_train, X_test, y_train, y_test, penalty,multiclass,classweight, seed)
            count += 1

            # output results to file
            row = [experiment, 'multiclass', 'Logistic Regression (sklearn)', X_train.shape, sample_length, coverage, k, penalty, multiclass, classweight, score, score_type]
            append_results_to_file(grid_search_file, row)
                        
        print('Percent complete: {}'.format(count / n_combinations * 100)) # display progress
    

### Run 7.01 - l1 penalty with saga solver
Can't use default solver with l1 penalty.

In [31]:
# %%time
# # 40 min

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/gridsearch-2000/lr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','penalty','multi_class', 'class_weight', 'score','score type']
# experiment = '7.01'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [100,200,400]
# list_coverage = [1,10,100,200,400]
# list_k = [1,2,4,6,8,10,12]
# list_penalty=['l1']
# list_multiclass = ['auto']
# list_classweight= [None]


# grid_search_multiclass_lr(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_penalty,
#                           list_multiclass,
#                           list_classweight,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)


### Run 7.02 - Balanced class weights
Set solver back to default.

In [32]:
# %%time
# 3 min

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/gridsearch-2000/lr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','penalty','multi_class', 'class_weight', 'score','score type']
# experiment = '7.02'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [100,200,400]
# list_coverage = [1,10,100,200,400]
# list_k = [1,2,4,6,8,10,12]
# list_penalty=['l2']
# list_multiclass = ['auto']
# list_classweight= ['balanced']


# grid_search_multiclass_lr(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_penalty,
#                           list_multiclass,
#                           list_classweight,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)


### Run 7.03 - No penalty, no class weights
Changed solver back to default.

In [33]:
# %%time
# # 5 min

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/gridsearch-2000/lr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','penalty','multi_class', 'class_weight', 'score','score type']
# experiment = '7.03'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [100,200,400]
# list_coverage = [1,10,100,200,400]
# list_k = [1,2,4,6,8,10,12]
# list_penalty=['none']
# list_multiclass = ['auto']
# list_classweight= [None]


# grid_search_multiclass_lr(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_penalty,
#                           list_multiclass,
#                           list_classweight,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)


### Run 7.04 - L2 penalty, ovr

In [34]:
# %%time
# 3min

# # parameters
# seq_file = 'data/train_small-db_toy-2000.fasta'
# taxid_file = 'data/train_small-db_toy-2000.taxid'
# output_dir = 'data/sampling/sampling-toy-2000'
# pattern = 'fragments*.npy'
# seed = 42
# date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
# grid_search_file  = 'data/gridsearch-2000/lr-multi.{}.csv'.format(date_time)
# fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','penalty','multi_class', 'class_weight', 'score','score type']
# experiment = '7.04'
# score_type = 'species_recall'

# # combinations to try
# list_sample_length = [100,200,400]
# list_coverage = [1,10,100,200,400]
# list_k = [1,2,4,6,8,10,12]
# list_penalty=['l2']
# list_multiclass = ['ovr']
# list_classweight= [None]


# grid_search_multiclass_lr(seq_file, 
#                           taxid_file, 
#                           output_dir, 
#                           pattern, 
#                           list_sample_length, 
#                           list_coverage, 
#                           list_k, 
#                           list_penalty,
#                           list_multiclass,
#                           list_classweight,
#                           seed,
#                           grid_search_file,
#                           fields,
#                           experiment,
#                           score_type)


# Run Set 9 - sklearn with l2 penalty and class weights on 7400 dataset

In [ ]:
%%time

# parameters
seq_file = 'data/train_small-db_toy-7400.fasta'
taxid_file = 'data/train_small-db_toy-7400.taxid'
output_dir = 'data/sampling/sampling-toy-7400'
pattern = 'fragments*.npy'
seed = 42
date_time = datetime.datetime.now().strftime('%Y.%m.%d.%H.%M.%S')
grid_search_file  = 'data/gridsearch-7400/results-lrpackage-l2-balanced.csv'
fields = ['experiment','category','classifier','training shape','sample_length','coverage','k','penalty','multi_class', 'class_weight', 'score','score type']
experiment = '9.01'
score_type = 'species_recall'

# combinations to try
list_sample_length = [100,200,400]
list_coverage = [1,10,100,200,400]
list_k = [1,2,4,6,8,10,12]
list_penalty=['l2']
list_multiclass = ['auto']
list_classweight= ['balanced']


grid_search_multiclass_lr(seq_file, 
                          taxid_file, 
                          output_dir, 
                          pattern, 
                          list_sample_length, 
                          list_coverage, 
                          list_k, 
                          list_penalty,
                          list_multiclass,
                          list_classweight,
                          seed,
                          grid_search_file,
                          fields,
                          experiment,
                          score_type)
